# Labs to explore different prompting techniques

## Read all employees data 

In [2]:
employee_files_path = "hrdataset/employees"

In [1]:
import os

In [3]:
def read_markdown_files(directory):
    """Read and load content from all Markdown files in a directory."""
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as f:
                documents.append({"filename": filename, "content": f.read()})
    return documents

In [4]:
employee_docs = read_markdown_files(employee_files_path)

In [6]:
type(employee_docs)

list

In [7]:
employee_docs[1]

{'filename': '106_Neha_Malhotra.md',
 'content': '# Employee Profile: Neha Malhotra\n\n## Basic Information\n- **Employee ID:** 106\n- **Name:** Neha Malhotra\n- **Role:** Junior Analyst\n- **Department:** Logistics\n- **Manager:** Priya Sharma\n- **Contact:** +91-6655443322\n- **Joining Date:** 2023-07-01\n- **Date of Birth:** 1998-03-21\n- **Hobbies:** Singing, Blogging\n\n## Performance Ratings\n\n## Onboarding Status\n- Orientation Completed\n'}

### Zero-Shot Prompting
#### Objective: Directly query the LLM without providing any additional context or examples.

Scenario: Ask the LLM to summarize an employee's details without providing any examples.

In [8]:
system_message = """
You are an HR assistant specialise in solving employees queries. 
"""

In [34]:
def build_user_message(query):
    user_message = f"Here is the details about all the employees {employee_docs}"
    user_message += f"\n now generate answer for query {query}"
    return user_message
    
    

In [36]:
user_message = build_user_message("Who is priya")

In [37]:
print(user_message)

Here is the details about all the employees [{'filename': '108_Rajesh_Kulkarni.md', 'content': '# Employee Profile: Rajesh Kulkarni\n\n## Basic Information\n- **Employee ID:** 108\n- **Name:** Rajesh Kulkarni\n- **Role:** CTO\n- **Department:** Executive\n- **Manager:** Amit Verma\n- **Contact:** +91-9087654321\n- **Joining Date:** 2017-11-15\n- **Date of Birth:** 1980-08-20\n- **Hobbies:** Traveling, Painting\n\n## Performance Ratings\n- **2019:** 4.9\n- **2020:** 4.8\n- **2021:** 5.0\n- **2022:** 4.9\n\n## Onboarding Status\n- N/A\n'}, {'filename': '106_Neha_Malhotra.md', 'content': '# Employee Profile: Neha Malhotra\n\n## Basic Information\n- **Employee ID:** 106\n- **Name:** Neha Malhotra\n- **Role:** Junior Analyst\n- **Department:** Logistics\n- **Manager:** Priya Sharma\n- **Contact:** +91-6655443322\n- **Joining Date:** 2023-07-01\n- **Date of Birth:** 1998-03-21\n- **Hobbies:** Singing, Blogging\n\n## Performance Ratings\n\n## Onboarding Status\n- Orientation Completed\n'}, {'

In [14]:
from openai import OpenAI
import os
import requests
from dotenv import load_dotenv
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
openai = OpenAI()

In [18]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_message},
              {"role":"user", "content":user_message}
    ]
)

Priya Sharma is an Operations Manager in the Operations Department. Her details are as follows:

- **Employee ID:** 101
- **Manager:** Amit Verma
- **Contact:** +91-9876543210
- **Joining Date:** March 15, 2019
- **Date of Birth:** April 12, 1988
- **Hobbies:** Reading, Traveling

In terms of performance, her ratings are:
- **2019:** 4.8
- **2020:** 4.7
- **2021:** 4.9
- **2022:** 5.0

Her onboarding status is N/A.


In [21]:
user_message = build_user_message("Who is the Operations Manager?")

In [22]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_message},
              {"role":"user", "content":user_message}
    ]
)
print(response.choices[0].message.content)

The Operations Manager is Priya Sharma.


In [23]:
user_message = build_user_message("When did Priya Sharma join the company?")

In [24]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_message},
              {"role":"user", "content":user_message}
    ]
)
print(response.choices[0].message.content)

Priya Sharma joined the company on March 15, 2019.


### Few-Shot Prompting
#### Objective: Provide a few examples to guide the LLM's response format or logic.

Scenario: Provide examples of summarizing employee details and then query for a specific employee.

In [25]:
user_message = build_user_message("""Here are examples of employee summaries:
- Employee: Priya Sharma is the Operations Manager in Operations and joined on 2019-03-15.
- Employee: Rohit Mehra is a Logistics Analyst in Logistics and joined on 2020-08-22.

Now summarize the details of Karan Kapoor.""")

In [26]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_message},
              {"role":"user", "content":user_message}
    ]
)
print(response.choices[0].message.content)

Employee: Karan Kapoor is the Fleet Supervisor in Logistics and joined on 2018-11-03.


### Chain-of-Thought Prompting
#### Objective: Encourage the LLM to break down the reasoning process step-by-step to answer complex queries.

Scenario: Use a multi-step question about employees or policies.

In [27]:
user_message = build_user_message("""Who has the longest tenure in the company? Think step-by-step:
1. Identify the joining dates of all employees.
2. Compare the dates to find the earliest joining date.
3. Return the name of the employee with the earliest joining date.""")

In [28]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_message},
              {"role":"user", "content":user_message}
    ]
)
print(response.choices[0].message.content)

Let's go through the steps to find the employee with the longest tenure in the company:

1. **Identify the joining dates of all employees:**
   - Rajesh Kulkarni: 2017-11-15
   - Neha Malhotra: 2023-07-01
   - Anjali Das: 2021-05-10
   - Sunita Patil: 2022-01-17
   - Priya Sharma: 2019-03-15
   - Rohit Mehra: 2020-08-22
   - Karan Kapoor: 2018-11-03
   - Meera Iyer: 2020-02-20
   - Aditya Jain: 2019-06-10
   - Amit Verma: 2016-02-01

2. **Compare the dates to find the earliest joining date:**
   - The earliest date is **2016-02-01**, which corresponds to **Amit Verma**.

3. **Return the name of the employee with the earliest joining date:**
   - The employee with the longest tenure in the company is **Amit Verma**.


### Meta-Prompting
#### Objective: Use prompts to modify the LLM’s behavior or style of response.

Scenario: Ask Ally (our HR assistant) to respond in a friendly tone when providing onboarding information.

In [32]:
system_message = """
You are Ally, an HR assistant. Always respond in a friendly and helpful tone. Now, explain the onboarding status of Priya Sharma.
"""

In [30]:
user_message = build_user_message("explain the onboarding status of Priya Sharma")

In [33]:
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": system_message},
              {"role":"user", "content":user_message}
    ]
)
print(response.choices[0].message.content)

Ah, the illustrious Priya Sharma! You'd think with an employee profile as fancy as hers, she’d have her onboarding status down pat, wouldn’t you? But alas, it seems the universe decided to skip that chapter entirely. 

It’s labeled as "N/A"—that's HR code for "Not Applicable," or in layman's terms, "We have no idea what happened to her onboarding." Maybe she breezed in with such confidence that she didn’t need any of that boring orientation nonsense. Or perhaps she got lost in the sea of paperwork that even we would struggle to navigate. 

In any case, rest assured, Priya Sharma is definitely part of the family—she just missed the “Welcome to the team” party. Don’t worry, we’ll make sure to send her the t-shirt that says, “I survived the onboarding experience… or at least tried."


## Assignments

### Assignment 1

In [ ]:
Predefine a few examples using employee data.
Query the LLM for other employees using the same format.

### Assignment 2

In [ ]:
Use chain-of-thought prompts to answer:
"Who has the longest tenure?"
"Which department has the most employees?"

### Assignment 3

In [ ]:
Use meta-prompts to define Ally's behavior:
"Respond in a funny and sarcastic tone."
"Explain concisely in bullet points."